# Function Generation for the Training of λ-Nets

## Specitication of Experiment Settings

In [1]:
#######################################################################################################################################
###################################################### CONFIG FILE ####################################################################
#######################################################################################################################################
sleep_time = 0 #minutes


config = {
    'function_family': {
        'maximum_depth': 4,
        'beta': 1,
        'decision_sparsity': 1,
        'fully_grown': True,                      
    },
    'data': {
        'number_of_variables': 30, 
        'num_classes': 2,
        
        'function_generation_type': 'make_classification_vanilla_decision_tree_trained', # 'make_classification' 'random_decision_tree' 'random_decision_tree_trained' 'random_vanilla_decision_tree_trained' 'make_classification_vanilla_decision_tree_trained'
        'objective': 'classification', # 'regression'
        
        'x_max': 1,
        'x_min': 0,
        'x_distrib': 'uniform', #'normal', 'uniform',       
                
        'lambda_dataset_size': 5000, #number of samples per function
        'number_of_generated_datasets': 10000,
    }, 
    'computation':{
        'n_jobs': -3,
        'use_gpu': False,
        'gpu_numbers': '0',
        'RANDOM_SEED': 0,   
    }
}


In [2]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['computation'])

## Imports

In [3]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from itertools import product       # forms cartesian products
from more_itertools import random_product 
from tqdm import tqdm_notebook as tqdm
import pickle

import numpy as np

import pandas as pd
from joblib import Parallel, delayed

import random 
from random import sample 

import os
import sys

from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler

from sympy import Symbol, sympify

        
import seaborn as sns
        
import random 

import warnings

from time import perf_counter



from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from utilities.DecisionTree_BASIC import SDT
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
from matplotlib import pyplot as plt
from IPython.display import Image

import numpy as np
import torch
np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

In [4]:
#######################################################################################################################################
###################################################### SET VARIABLES + DESIGN #########################################################
#######################################################################################################################################
sns.set_style("darkgrid")

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
    
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)

warnings.filterwarnings('ignore')

In [5]:
from utilities.utility_functions import *
#######################################################################################################################################
####################################################### CONFIG ADJUSTMENTS ############################################################
#######################################################################################################################################

#######################################################################################################################################
################################################## UPDATE VARIABLES ###################################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['computation'])

#initialize_utility_functions_config_from_curent_notebook(config)

#######################################################################################################################################
###################################################### PATH + FOLDER CREATION #########################################################
#######################################################################################################################################
globals().update(generate_paths(config, path_type='data_creation'))
generate_directory_structure()

#######################################################################################################################################
############################################################ SLEEP TIMER ##############################################################
#######################################################################################################################################
sleep_minutes(sleep_time)

In [6]:
print(path_identifier_function_data)

lNetSize5000_numDatasets10000_var30_class2_make_classification_vanilla_decision_tree_trained_xMax1_xMin0_xDistuniform_depth4_beta1_decisionSpars1_fullyGrown


# Function Generation

In [7]:
if function_generation_type == 'random_decision_tree':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_decision_tree_data)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))
elif function_generation_type == 'random_decision_tree_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_decision_tree_data_trained)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))  
elif function_generation_type == 'make_classification':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_decision_tree_data_trained_make_classification)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets)) 
elif function_generation_type == 'make_classification_vanilla_decision_tree_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_decision_tree_data_trained_make_classification_vanilla_decision_tree_trained)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))    
elif function_generation_type == 'random_vanilla_decision_tree_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_decision_tree_data_random_vanilla_decision_tree_trained)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets)) 

function_identifier_list = generate_decision_tree_identifier(config)  
identifier_series_list = [pd.Series(result[0],  index=function_identifier_list) for result in result_list]

function_df = pd.DataFrame(data=np.array([result[0] for result in result_list]), columns=function_identifier_list)

X_data_list = [[identifier_series, pd.DataFrame(result[1], columns=['x' + str(i) for i in range(number_of_variables)])] for identifier_series, result in zip(identifier_series_list, result_list)]
y_data_list = [[identifier_series, pd.DataFrame(result[2], columns=['result'])] for identifier_series, result in zip(identifier_series_list, result_list)]
y_data_raw_list = [[identifier_series, pd.DataFrame(result[3], columns=['result_raw'])] for identifier_series, result in zip(identifier_series_list, result_list)]


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-3)]: Done 1108 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-3)]: Done 1524 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-3)]: Done 2004 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-3)]: Done 2548 tasks      | elapsed:   22.2s
[Parallel(n_jobs=-3)]: Done 3156 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-3)]: Done 3828 tasks      | elapsed:   29.0s
[Parallel(n_jobs=-3)]: Done 4564 tasks      | elapsed:   32.9s
[Parallel(n_jobs=-3)]: Done 5364 tasks      | elapsed:   37.1s
[Parallel(n_jobs=-3)]: Done 6228 tasks      | elapsed:   41.7s
[Parallel(n_jobs=-3)]: Done 7156 tasks      | elapsed:   46.6s
[Parallel(n_jobs=-3)]: Done 8148 tasks      |

In [8]:
function_df.head()

,f0v0,f0v1,f0v2,f0v3,f0v4,f0v5,f0v6,f0v7,f0v8,f0v9,f0v10,f0v11,f0v12,f0v13,f0v14,f0v15,f0v16,f0v17,f0v18,f0v19,f0v20,f0v21,f0v22,f0v23,f0v24,f0v25,f0v26,f0v27,f0v28,f0v29,f1v0,f1v1,f1v2,f1v3,f1v4,f1v5,f1v6,f1v7,f1v8,f1v9,f1v10,f1v11,f1v12,f1v13,f1v14,f1v15,f1v16,f1v17,f1v18,f1v19,f1v20,f1v21,f1v22,f1v23,f1v24,f1v25,f1v26,f1v27,f1v28,f1v29,f2v0,f2v1,f2v2,f2v3,f2v4,f2v5,f2v6,f2v7,f2v8,f2v9,f2v10,f2v11,f2v12,f2v13,f2v14,f2v15,f2v16,f2v17,f2v18,f2v19,f2v20,f2v21,f2v22,f2v23,f2v24,f2v25,f2v26,f2v27,f2v28,f2v29,f3v0,f3v1,f3v2,f3v3,f3v4,f3v5,f3v6,f3v7,f3v8,f3v9,f3v10,f3v11,f3v12,f3v13,f3v14,f3v15,f3v16,f3v17,f3v18,f3v19,f3v20,f3v21,f3v22,f3v23,f3v24,f3v25,f3v26,f3v27,f3v28,f3v29,f4v0,f4v1,f4v2,f4v3,f4v4,f4v5,f4v6,f4v7,f4v8,f4v9,f4v10,f4v11,f4v12,f4v13,f4v14,f4v15,f4v16,f4v17,f4v18,f4v19,f4v20,f4v21,f4v22,f4v23,f4v24,f4v25,f4v26,f4v27,f4v28,f4v29,f5v0,f5v1,f5v2,f5v3,f5v4,f5v5,f5v6,f5v7,f5v8,f5v9,f5v10,f5v11,f5v12,f5v13,f5v14,f5v15,f5v16,f5v17,f5v18,f5v19,f5v20,f5v21,f5v22,f5v23,f5v24,f5v25,f5v26,f5v27,f5v28,f5v29,f6v0,f6v1,f6v2,f6v3,f6v4,f6v5,f6v6,f6v7,f6v8,f6v9,f6v10,f6v11,f6v12,f6v13,f6v14,f6v15,f6v16,f6v17,f6v18,f6v19,f6v20,f6v21,f6v22,f6v23,f6v24,f6v25,f6v26,f6v27,f6v28,f6v29,f7v0,f7v1,f7v2,f7v3,f7v4,f7v5,f7v6,f7v7,f7v8,f7v9,f7v10,f7v11,f7v12,f7v13,f7v14,f7v15,f7v16,f7v17,f7v18,f7v19,f7v20,f7v21,f7v22,f7v23,f7v24,f7v25,f7v26,f7v27,f7v28,f7v29,f8v0,f8v1,f8v2,f8v3,f8v4,f8v5,f8v6,f8v7,f8v8,f8v9,f8v10,f8v11,f8v12,f8v13,f8v14,f8v15,f8v16,f8v17,f8v18,f8v19,f8v20,f8v21,f8v22,f8v23,f8v24,f8v25,f8v26,f8v27,f8v28,f8v29,f9v0,f9v1,f9v2,f9v3,f9v4,f9v5,f9v6,f9v7,f9v8,f9v9,f9v10,f9v11,f9v12,f9v13,f9v14,f9v15,f9v16,f9v17,f9v18,f9v19,f9v20,f9v21,f9v22,f9v23,f9v24,f9v25,f9v26,f9v27,f9v28,f9v29,f10v0,f10v1,f10v2,f10v3,f10v4,f10v5,f10v6,f10v7,f10v8,f10v9,f10v10,f10v11,f10v12,f10v13,f10v14,f10v15,f10v16,f10v17,f10v18,f10v19,f10v20,f10v21,f10v22,f10v23,f10v24,f10v25,f10v26,f10v27,f10v28,f10v29,f11v0,f11v1,f11v2,f11v3,f11v4,f11v5,f11v6,f11v7,f11v8,f11v9,f11v10,f11v11,f11v12,f11v13,f11v14,f11v15,f11v16,f11v17,f11v18,f11v19,f11v20,f11v21,f11v22,f11v23,f11v24,f11v25,f11v26,f11v27,f11v28,f11v29,f12v0,f12v1,f12v2,f12v3,f12v4,f12v5,f12v6,f12v7,f12v8,f12v9,f12v10,f12v11,f12v12,f12v13,f12v14,f12v15,f12v16,f12v17,f12v18,f12v19,f12v20,f12v21,f12v22,f12v23,f12v24,f12v25,f12v26,f12v27,f12v28,f12v29,f13v0,f13v1,f13v2,f13v3,f13v4,f13v5,f13v6,f13v7,f13v8,f13v9,f13v10,f13v11,f13v12,f13v13,f13v14,f13v15,f13v16,f13v17,f13v18,f13v19,f13v20,f13v21,f13v22,f13v23,f13v24,f13v25,f13v26,f13v27,f13v28,f13v29,f14v0,f14v1,f14v2,f14v3,f14v4,f14v5,f14v6,f14v7,f14v8,f14v9,f14v10,f14v11,f14v12,f14v13,f14v14,f14v15,f14v16,f14v17,f14v18,f14v19,f14v20,f14v21,f14v22,f14v23,f14v24,f14v25,f14v26,f14v27,f14v28,f14v29,b0,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,b11,b12,b13,b14,lp0c0,lp0c1,lp1c0,lp1c1,lp2c0,lp2c1,lp3c0,lp3c1,lp4c0,lp4c1,lp5c0,lp5c1,lp6c0,lp6c1,lp7c0,lp7c1,lp8c0,lp8c1,lp9c0,lp9c1,lp10c0,lp10c1,lp11c0,lp11c1,lp12c0,lp12c1,lp13c0,lp13c1,lp14c0,lp14c1,lp15c0,lp15c1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [9]:
X_data_list[0][0].head(10)

f0v0    0
f0v1    0
f0v2    0
f0v3    0
f0v4    0
f0v5    0
f0v6    0
f0v7    0
f0v8    0
f0v9    0
dtype: int64

In [10]:
X_data_list[0][1].head(10)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29
0,0.364,0.665,0.530,0.500,0.531,0.139,0.544,0.381,0.425,0.541,0.663,0.450,0.467,0.465,0.267,0.544,0.553,0.384,0.453,0.649,0.461,0.473,0.308,0.598,0.497,0.410,0.661,0.294,0.518,0.805
1,0.661,0.322,0.451,0.489,0.323,0.424,0.442,0.386,0.547,0.722,0.312,0.447,0.451,0.615,0.620,0.378,0.600,0.423,0.249,0.558,0.586,0.175,0.401,0.332,0.685,0.533,0.524,0.362,0.341,0.344
2,0.573,0.404,0.373,0.315,0.455,0.639,0.520,0.463,0.504,0.420,0.574,0.362,0.519,0.519,0.397,0.543,0.420,0.421,0.461,0.577,0.462,0.526,0.413,0.545,0.463,0.387,0.367,0.492,0.561,0.411
3,0.600,0.339,0.448,0.482,0.524,0.392,0.511,0.440,0.657,0.873,0.473,0.574,0.646,0.442,0.442,0.627,0.581,0.388,0.498,0.589,0.470,0.382,0.569,0.579,0.647,0.349,0.535,0.390,0.397,0.521
4,0.304,0.517,0.538,0.431,0.403,0.620,0.399,0.482,0.473,0.420,0.428,0.434,0.480,0.445,0.448,0.667,0.498,0.504,0.407,0.342,0.545,0.654,0.296,0.518,0.692,0.698,0.435,0.412,0.481,0.512
5,0.378,0.583,0.483,0.381,0.477,0.430,0.600,0.564,0.603,0.432,0.464,0.310,0.417,0.416,0.576,0.468,0.492,0.427,0.219,0.368,0.675,0.761,0.472,0.598,0.596,0.318,0.630,0.551,0.409,0.517
6,0.310,0.520,0.572,0.439,0.409,0.440,0.276,0.487,0.850,0.276,0.455,0.594,0.326,0.844,0.612,0.426,0.497,0.417,0.485,0.361,0.533,0.322,0.464,0.629,0.365,0.311,0.603,0.506,0.314,0.426
7,0.484,0.571,0.481,0.418,0.517,0.499,0.514,0.549,0.453,0.354,0.673,0.521,0.600,0.658,0.572,0.536,0.569,0.379,0.377,0.573,0.716,0.555,0.340,0.468,0.627,0.529,0.644,0.772,0.377,0.489
8,0.331,0.394,0.077,0.500,0.515,0.437,0.699,0.413,0.436,0.554,0.537,0.453,0.794,0.507,0.631,0.548,0.455,0.498,0.659,0.434,0.531,0.490,0.540,0.334,0.554,0.642,0.609,0.571,0.605,0.510
9,0.410,0.559,0.274,0.492,0.683,0.575,0.466,0.519,0.671,0.458,0.534,0.583,0.612,0.281,0.390,0.424,0.635,0.366,0.411,0.241,0.630,0.388,0.577,0.493,0.635,0.349,0.238,0.564,0.503,0.639


In [11]:
y_data_list[0][0].head(10)

f0v0    0
f0v1    0
f0v2    0
f0v3    0
f0v4    0
f0v5    0
f0v6    0
f0v7    0
f0v8    0
f0v9    0
dtype: int64

In [12]:
y_data_list[2][1].head(10)

,result
0,0
1,0
2,1
3,0
4,1
5,0
6,1
7,1
8,1
9,1


In [13]:
y_data_raw_list[2][1].head(10)

,result_raw
0,0
1,0
2,1
3,0
4,1
5,0
6,1
7,1
8,1
9,1


In [14]:
#some_tree = generate_decision_tree_from_array(y_data_list[0][0].values, config)

In [15]:
#some_tree.plot_tree()


In [16]:
path_functions = './data/saved_function_lists/functions_' + path_identifier_function_data + '.csv'
function_df.to_csv(path_functions, index=False)

path_X_data = './data/saved_function_lists/X_data_' + path_identifier_function_data + '.pkl'
with open(path_X_data, 'wb') as f:
    pickle.dump(X_data_list, f)
    
path_y_data = './data/saved_function_lists/y_data_' + path_identifier_function_data + '.pkl'
with open(path_y_data, 'wb') as f:
    pickle.dump(y_data_list, f)
